In [1]:
from collections import Counter
import matplotlib.pyplot as plt
from ProductListingParsingTools import *

In [ ]:
# example for synthetic pages
directory = "./data/agora_cat/2014-04-06/cat/"
productsDF = directoryToDF()
productsDF.to_csv("catProducts.csv")

./data/agora_cat/2014-03-03/cat/.DS_Store
Error not found: ./data/agora_cat/2014-03-03/cat/.DS_Store
./data/agora_cat/2014-01-01/cat/.DS_Store
Error not found: ./data/agora_cat/2014-01-01/cat/.DS_Store


In [ ]:
productsDF.count()

In [ ]:
productsDF.head()